In [ ]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import numpy as np

In [ ]:
# Read the dataset
data = pd.read_csv("data.txt", delimiter=" ", header=None, names=["node1", "node2", "weight"])

# Create a graph
G = nx.Graph()

In [ ]:
# Add edges with weight attribute
for idx, row in data.iterrows():
    G.add_edge(row["node1"], row["node2"], weight=row["weight"])

# Calculate some network metrics
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
density = nx.density(G)
degree_distribution = dict(G.degree())
average_degree = np.mean(list(degree_distribution.values()))

In [ ]:
# Print metrics
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Density: {density}")
print(f"Average degree: {average_degree}")

# Perform ERGM
# ERGM is currently not supported directly in NetworkX, but you can use the R package `statnet` with the `rpy2` package to call R functions from Python
# Installation: pip install rpy2
# In R: install.packages('statnet')

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter

In [ ]:
statnet = importr("statnet")
pandas2ri.activate()

# Convert the NetworkX graph to an R 'network' object
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(data)

In [ ]:
r_edges = r("as.matrix")(r_data)
r_graph = statnet.network(r_edges, matrix_type="edgelist", directed=False)

# Perform ERGM
ergm_model = statnet.ergm(r_graph ~ statnet.edges() + statnet.nodematch("weight"))
print(ergm_model)